# Семинар 2 - Представление данных

Сегодня в меню:
* Выравнивания
* Aliasing
* Целочисленные типы данных
* Типы данных с плавающей точкой
* Кодировки

## Общее введение

### Выравнивания

Минимально адресуемой единицей является байт. Сколько бит в байте? Ответ содержится в константе `CHAR_BIT` из `limits.h`. По стандарту гарантируется, что хотя бы 8. Однобайтовый тип данных называется `char` (то есть `sizeof(char) == 1`).

Значит ли это, что под `char` всегда будет выделяться ровно 1 байт? Не совсем

<img src="bool_is_1_bit.jpg" alt="bool is only one bit" width="500"/>

Современные архитектуры компьютеров устроены таким образом, что обращения к данным происходят наиболее быстрым образом, когда данные *натурально выравнены*. Это значит, что их адрес в памяти кратен их выравниваю. Выравнивание, в свою очередь, определяется так:
* `alignof(T) == sizeof(T)` для примитивных типов (`int`, `float`, `char`, `T*` и т. д.)
* Если `T` - структура, то `alignof(T)` равен максимальному `alignof` среди её полей

<img src="alignment_perf.jpg" alt="alignment perf" width="500"/> <img src="4alignment_perf.jpg" alt="alignment perf" width="496"/>


Время доступа к 1-, 2- и 4-байтовым объектам в зависимости от выравнивания.

Почему доступ к 4-байтовым объектам быстрее? Из-за того, что измерения проводились на комьпьютере с 4-байтовым [словом](https://en.wikipedia.org/wiki/Word_(computer_architecture))

По умолчанию компиляторы C располагают все объекты натурально выровненными, в том числе все поля структур. Чтобы этого добиться, между полями структуры лежат "padding bytes" - байты, которые не хранят в себе информации и служат лишь для соблюдения выравниваний полей:

In [48]:
!cat -n padding.c
!gcc padding.c -o padding.out

     1	#include <stddef.h>
     2	#include <stdio.h>
     3	#include <stdlib.h>
     4	
     5	int main(void) {
     6	    struct s {
     7	        int i;    // 4 bytes
     8	        char c;   // 1 byte
     9	        double d; // 8 bytes
    10	        char a;   // 1 byte
    11	    };
    12	
    13	    /* Output is compiler dependent */
    14	
    15	    printf("offsets: i=%ld; c=%ld; d=%ld a=%ld\n",
    16	        (long) offsetof(struct s, i),
    17	        (long) offsetof(struct s, c),
    18	        (long) offsetof(struct s, d),
    19	        (long) offsetof(struct s, a));
    20	    printf("sizeof(struct s)=%ld\n", (long) sizeof(struct s));
    21	}

In [4]:
!./padding.out 

offsets: i=0; c=4; d=8 a=16
sizeof(struct s)=24


То есть в памяти структура `s` располагается таким образом:

```c
struct s {
    int i;    // 4 bytes
    char c;   // 1 byte
              // 3 byte padding, so &d % 8 == 0
    double d; // 8 bytes
    char a;   // 1 byte
              // 7 byte padding for the whole structure
};
```

> В конце структуры тоже бывает паддинг, чтобы `sizeof(T)` был кратен `alignof(T)`. Зачем?
<details>
  <summary>Ответ</summary>
  Чтобы при хранении структур в массиве (T[]) все объекты были выровнены и лежали в памяти непрерывно. 
</details>

Если менять порядок полей внутри структуры, можно влиять на количество паддинга и менять `sizeof` всей структуры. С гарантирует, что поля будут лежать в памяти в том же порядке, в котором вы их перечислили, то есть без переставлений. Rust, для сравнения, по умолчанию будет искать оптимальный порядок полей, чтобы минимизировать `sizeof` структуры с сохранением выравниваний полей.

Можно [дать указание](https://www.ibm.com/docs/en/zos/2.4.0?topic=descriptions-pragma-pack) компилятору не добавлять padding bytes и упаковать все поля друг за другом. Использовать такие хаки надо с очень большой осторожностью.

> Вопрос: гарантируется ли выраванивание при динамической аллокации памяти через `malloc`/`new`? Если да, как это достигается?
<details>
  <summary>Ответ</summary>
  
  Да, гарантируется. Динамически аллоцированная память выровнена на `alignof(maxalign_t)`, то есть имеет максимальное выравнивание, которое подойдет всем объектам. Также есть `void *aligned_alloc(size_t alignment, size_t size);`
  
</details>

#### Больше о выравнивании:
* [The lost Art of Structure Packing](http://www.catb.org/esr/structure-packing/)
* [Data alignment: Straighten up and fly right](https://developer.ibm.com/articles/pa-dalign/)

### Aliasing 

Иногда возникает необходимость интепретировать какой-то объект типа `T` как объект типа `U`. Такое действие называется type punning. К примеру, далее мы будем смотреть, как в памяти размещены объекты типа `float`, для этого нужно будет интепретировать их как `char[]`, чтобы посмотреть на составляющие байты. 

Если `T` и `U` достаточно "разные", то такая интерпретация почти всегда будет являться undefined behaviour. Это вызвано нарушением strict aliasing rules.

Как делать type punning, чтобы это не вызывало ub?
* Кастовать к `char*` можно всегда (*)
* Использовать `memcpy`
* Использовать `union` (только в C)
* Использовать `std::bit_cast<>` (только в C++)

Избегайте type punning через указатели.

In [5]:
!cat -n type_punning.c 

     1	uint32_t test1(uint32_t arg) {
     2	    union {
     3	        uint32_t arg;
     4	        uint16_t arg2[2];
     5	    }c = {.arg = arg};
     6	
     7	    c.arg2[0] = 0x1;
     8	    c.arg2[1] = 0x1;
     9	    return (c.arg);
    10	}
    11	
    12	uint32_t test2(uint32_t arg) {
    13	    unsigned char *ptr = &arg;
    14	    memcpy(ptr, (uint16_t[]){1}, sizeof(uint16_t));
    15	    memcpy(ptr + 2, (uint16_t[]){1}, sizeof(uint16_t));
    16	    return arg;
    17	}
    18	
    19	uint32_t test3(uint32_t arg) {
    20	    uint16_t* const sp = (uint16_t*)&arg;  // dereferencing type-punned pointer will break strict-aliasing rules (ub)
    21	    uint16_t*       hi = &sp[0];
    22	    uint16_t*       lo = &sp[1];
    23	
    24	    *hi = 0x1;
    25	    *lo = 0x1;
    26	
    27	    return arg;
    28	} 


#### Подробнее об aliasing rules:
* [Understanding Strict Aliasing](https://cellperformance.beyond3d.com/articles/2006/06/understanding-strict-aliasing.html) - с vpn
* [Demystifying The Restrict Keyword](https://cellperformance.beyond3d.com/articles/2006/05/demystifying-the-restrict-keyword.html) - с vpn
* [What is the Strict Aliasing Rule and Why do we care?](https://gist.github.com/shafik/848ae25ee209f698763cffee272a58f8)

## Целые числа

Какие есть целочисленные типы в C?

<img src="arithmetic_types.png" alt="arithmetic types" width="500"/> <img src="fixed_types.png" alt="arithmetic types" width="528"/>

Аргументы в пользу типов фиксированного размера в принципе понятны. Аргументы против [тоже есть](https://clownacy.wordpress.com/2023/07/07/stop-using-fixed-width-integer-types/)

Производя `+`, `-`, `*` со стандартными целочисленными типами в программе мы работаем $\mathbb{Z}_{2^k}$, где $k$ - количество бит в числе. Причем это верно как со знаковыми, так и беззнаковыми числами.

В процессоре для сложения знаковых и беззнаковых чисел выполняется одна и та же инструкция.

Но есть некоторые тонкости.



Если вы пишете код на C/C++ то компилятор считает переполнение **знакового** типа UB. Это позволяет ему проводить оптимизации.

А переполнение беззнакового типа - законной операцией, при которой просто отбрасываются старшие биты (или значение берется по модулю $2^k$, или просто операция производится в $\mathbb{Z}_{2^k}$ - можете выбирать более удобный для вас способ на это смотреть).

In [9]:
!cat -n increment.c | awk 'NR < 9'

     1	int check_increment(int x) {
     2	    return x + 1 > x; // Всегда ли true?
     3	}
     4	
     5	int unsigned_check_increment(unsigned int x) {
     6	    return x + 1 > x; // Всегда ли true?
     7	}
     8	


In [17]:
!gcc -O3 increment.c -o increment.out
!./increment.out

increment.c:14:38: warning: overflow in expression; result is -2'147'483'648 with type 'int' [-Winteger-overflow]
   14 |     printf("x + 1 = %d\n", INT32_MAX + 1);
      |                            ~~~~~~~~~~^~~
1 warning generated.
x =     2147483647
x + 1 = -2147483648
x + 1 > x, for int x  = INT32_MAX:  1


x =     4294967295
x + 1 = 0
x + 1 > x, for uint x = UINT32_MAX: 0


Посмотрим непосредственно на то, как целые числа лежат в памяти

In [49]:
!gcc print_int.c -o print_int.out
!./print_int.out

uint8_t  27: 00011011 
int8_t   27: 00011011 
int8_t  -27: 11100101 
uint16_t 27 (little endian): 00011011 00000000 
uint16_t 27    (big endian): 00000000 00011011 


Целые числа хранятся в соответствии с 2's complement: 
* Первый бит отвечает за знак
* 0 имеет единственное представление - 000...00
* Из-за того, что 0 получается "положительным", отрицательных чисел можно представить на 1 больше
* Отрицательные числа хранятся в инвертированном виде (-1 это 11111111, а не 10000001)

Подробнее о последнем пункте: это сделано, чтобы для при реализации сложения не приходилось отдельно обрабатывать случай отрицательных чисел. К примеру, -1 + 1 = 0 получается ествественным образом: `11111111 + 00000001 = 00000000`

<img src="2complement.png" alt="2's complement" width="250"/>

Минус такого представления: чему равно `-INT_MIN`?

В чем проблема "естественного" представления со знаковым битом и содержательной частью? Получается 2 нуля

Endianess определяет порядок **байт** в представлении числа. При little endian сначала идут младшие байты, затем старшие. Порядок бит внутри каждого байта при этом сохраняется.

`uint16_t 27    (big endian): 00011011 00000000`

В большинстве компьютеров используется little endian. Наиболее частый случай использования big endian - передача данных по сети

> Как проверить endianess компьютера, на котором исполняется программа?

<details>
  <summary>Ответ</summary>

  ```c
  int is_little_endian() {
    int   one  = 1;
    char* byte = (char*)&one;
    return byte[0] == 1;
  }
  ```

</details>


## Числа с плавающей точкой

Стандарт C не дает гарантий о внутреннем представлении чисел с плавающей точкой, но в подавляющем большинстве случаев используется стандарт IEEE-754. Как он устроен? Посмотрим на примере 32-битного типа

<img src="ieee754.png" alt="ieee-754" width="900" style="background-color:white;"/>

Биты числа разделены на 3 группы:

1. **Знаковый бит.** Один, старший бит. 0 – число положительное (или нуль), 1 – отрицательное или нуль.
2. **Показатель степени.** 8-битное целое число, занимает биты с 23-го по 30-ый. Означает степень двойки, на которую будет домножаться основаная часть числа, записанная далее.
3. **Дробная часть.** 23-битное целое число, содержащее значащие биты вещественного числа.

Обозначим знаковый бит как `sign`, беззнаковое значение показателя степени как `exp`, а беззнаковое значение дробной части как `frac`.

### Значение числа в разных случаях

Итак, как числа получаются с помощью этих компонент? Есть несколько случаев:
1. Если $0 < exp < 255$, число называется нормализованным. В таком случае оно равно $−1^{sign}\times2^{exp − 127}\times1,frac_2$, то есть знак и степень двойки домножаются на число, у которого в целой части стоит 1, а двоичную дробную часть составляют 23 бита `frac`.

Т. к. мы хотим представлять как большие по модулю числа, так и близкие к нулю, то степень двойки должна принимать как положительные, так и отрицательные значения. Поэтому мы вычитаем из неё 127 (смещение), так степень станет принимать значения [-126; 127] (почему не могут быть -127 и 128, увидим ниже).

<img src="normalized.png" alt="normalized ieee-754" width="900"/>

2. Если `exp=0`, число называется денормализованным. В этом случае его значение $−1^{sign}\times2^{−126}\times0,frac_2$.

<img src="denormalized.png" alt="denormalized ieee-754" width="900"/>

3. Если `exp=255` и `frac=0`, число называется бесконечностью и обозначается `+inf` или `-inf` в зависимости от знака.

<img src="inf.png" alt="inf ieee-754" width="900"/>

Бесконечности нужны для обозначения очень больших по модулю чисел, для представления которых не достаточно 8 бит показателя степени.

4. Если `exp=255` и `frac!=0`, то это `"NaN"`. Знак при этом ни на что не влияет.

<img src="nan.png" alt="nan ieee-754" width="900"/>

Как видно, различных `NaN`-ов бывает много. Все они нужны для обозначения неопределённости в результате выполнения арифметических операций, например, `inf-inf` или `sqrt(-3)`.


Итого, числовая прямая выглядит так:

<img src="line.png" alt="ieee-754 line" width="900"/>


### Особенности формата

* Есть два нуля: +0 и -0. +0 имеет все нулевые биты, то есть является нулём как 32-битное целое число.

* **Сравнение**: наименьшее положительное нормализованное число ($2^{−126}\times1,0$) больше наибольшего денормализованного числа ($2^{−126}\times0,111..1$). Также любое положительное нормализованное число с меньшим $exp$ меньше любого положительного нормализованного числа с большим $exp$. Таким образом, действительно, все положительные денормализованные числа меньше всех положительных нормализованных, а нормализованные расположены в порядке показателя степени.

* Такой же порядок остаётся, если сравнивать битовые представления вещественных чисел как знаковые целые числа! (Кроме NaN-ов).

* Вещественные числа **становятся более разреженными** при увеличении их модуля. Чем число ближе к нулю, тем оно ближе к ближайшему к нему другому представимому числу. А точнее, денормализованные числа и нормализованные с `exp=1` идут через одинаковый шаг. Нормализованные числа с `exp=2` идут через удвоенный шаг, с `exp=3` - через учетверённый, и так далее. Иллюстрация распределения чисел:

<img src="distribution.png" alt="ieee-754 numbers distribution" width="900"/>


### Арифметические операции
* **Умножение** числа на **степень двойки**. Достаточно прибавить/вычесть из показателя степени (учитывая переполнения).
* **Умножение** двух чисел. Достаточно **сложить их показатели степени и перемножить мантиссы**. При этом в мантиссе может также получиться число >= 2, тогда надо его нормализовать и прибавить 1 к показателю степени. **Если мантисса** результата не влезает в 23 бита, её надо **округлить**. Если хвост < 1/2 или > 1/2, то округляется вниз или вверх, соответственно. Если хвост в точности 1/2, то округлять всегда в одну сторону плохо, т.к. при большом количестве последовательных операций может накопиться существенная погрешность. Поэтому в этом случае округляется к ближайшему чётному (то есть, если предыдущий бит равен 0, то вниз, если 1 -- вверх). Пример:


* **Деление** аналогично.
* **Сложение и вычитание**. Привести оба числа к одинаковому показателю степени, выполнить операцию, привести к нормализованному или денормализованному виду и округлить по правилам выше, если потребуется.

### Свойства арифметических операций
* При корректных арифметических операциях получается либо число, либо бесконечность.

* Сложение и умножение коммутативны

* Ассоциативности нет. Из-за округления в процессе выполнения операций могут получиться разные результаты. Например, $(3.14+2^{100})−2^{100}=0$, но $3.14+(2^{100}−2^{100})=3.14$. Из-за этого, для сохранения точности выполнять операции стоит в определённом порядке. Например, если хотим сложить массив вещественных чисел в одно число, лучше всего делать это в порядке сортировки чисел по возрастанию модулей.

* У конечных чисел есть обратный элемент по сложению.

* Монотонность: $a \geq b \Rightarrow a+c \geq b+c$, если нет переполнений и NaN-ов.



### Больше про IEEE-754:
* [Калькулятор](https://www.h-schmidt.net/FloatConverter/IEEE754.html)
* [Визуализатор арифметики](https://weitz.de/ieee/)
* Где множество чисел с плавающей точкой, оснащенное операциями сложения и умножения, находится в алгебраической иерархии? (множество, магма, полугруппа, группа, кольцо, поле) <details> <summary>Ответ</summary> Является множеством и магмой. Полугруппой уже не является, так как нет ассоциативности сложения</details>
* Влияет ли endianess на представление чисел с плавающей точкой в памяти? <details>
    <summary>Ответ</summary>
    Да, влияет. Пример в float_endianess.c
</details>

## Кодировки

<img src="ascii.png" alt="ASCII" width="900" style="background-color:white;"/>

В середине 20 века возникла потребность как-то кодировать текст двоичными данными. Тогда решили, что 7 бит достаточно на цифры и латинский алфавит, так и появилась кодировка ASCII.

Позже один лишний бит позволил хранить в диапазоне от 128 до 255 символы национальных кодировок. В России, например, в 90-е была распространена `KOI8-R`.

Иметь отдельную кодировку под каждый язык всё ещё было не очень удобно, поэтому появилась `UTF-8`. Чтобы не занимать гарантированно 4 байта (октета), был придуман интересный дизайн:

| Количество октетов | Шаблон |
|---|---|
|1|0xxxxxxx|
|2|110xxxxx 10xxxxxx|
|3|1110xxxx 10xxxxxx 10xxxxxx|
|4|11110xxx 10xxxxxx 10xxxxxx 10xxxxxx|

Таким образом:
1. UTF-8 совместима с ASCII, то есть кодирует символы точно так же 
2. Имеет нефиксированное количество байт на символ, тем самым позволяя кодировать более часто используемые символы меньшим количество байт
3. Не позволяет делать быстрый random access (доступ к n-му символу строки)
4. Не позволяет быстро узнать количество символов в строке

<img src="semicolon_meme.png" alt="semicolon meme" width="500"/>

In [58]:
def add_spaces(s):
    return "".join(c + ("" if (i + 1) % 6 else " ") for i, c in enumerate(s[::-1]))[::-1]

def show_utf_8(c):
    num = c if isinstance(c, int) else ord(c) 
    print("       CHR:", chr(num))
    encoded = chr(num).encode("utf-8")
    print("   BIN NUM: {:b}".format(num))
    print("  BIN NUM2:", add_spaces("{:b}".format(num)))
    print(" UTF-8 BIN:", " ".join("{:b}".format(b).rjust(8, '0') for b in encoded))

In [59]:
show_utf_8("q")

       CHR: q
   BIN NUM: 1110001
  BIN NUM2: 1 110001
 UTF-8 BIN: 01110001


In [60]:
show_utf_8("\n")

       CHR: 

   BIN NUM: 1010
  BIN NUM2: 1010
 UTF-8 BIN: 00001010


In [61]:
show_utf_8("ы")

       CHR: ы
   BIN NUM: 10001001011
  BIN NUM2: 10001 001011
 UTF-8 BIN: 11010001 10001011


In [62]:
show_utf_8("岁")

       CHR: 岁
   BIN NUM: 101110010000001
  BIN NUM2: 101 110010 000001
 UTF-8 BIN: 11100101 10110010 10000001


In [63]:
show_utf_8("😎")

       CHR: 😎
   BIN NUM: 11111011000001110
  BIN NUM2: 11111 011000 001110
 UTF-8 BIN: 11110000 10011111 10011000 10001110


UTF-8 - это кодировка, определяющая, какими байтами будет записан один Unicode codepoint. Некоторые символы записываются несколькими codepoint-ами (composite codepoint):

In [64]:
show_utf_8("n̂")

TypeError: ord() expected a character, but string of length 2 found

In [65]:
s = "n̂"
show_utf_8(s[0])
print()
show_utf_8(s[1])

       CHR: n
   BIN NUM: 1101110
  BIN NUM2: 1 101110
 UTF-8 BIN: 01101110

       CHR: ̂
   BIN NUM: 1100000010
  BIN NUM2: 1100 000010
 UTF-8 BIN: 11001100 10000010


### Как это ложится на языки программирования?
* C: строки - просто `char*`, то есть последовательность байт. Кодировки - более высокоуровневые абстракции
* Rust: строки хранятся в UTF-8 совместимом представлении. Последствия:
```rust
let s = String::from("ラウトは難しいです！");
s.len() // returns 30 (number of bytes).
s[0] // does not compile: the type `std::string::String` cannot be indexed by `usize`
``` 
* Python: [все сложно](https://dev.to/bplevin36/python-strings-are-not-utf-8-2dfj) (не UTF-8, но это не так легко заметить)

Больше о хранении символов:
* [Объяснение терминов](https://stackoverflow.com/questions/27331819/whats-the-difference-between-a-character-a-code-point-a-glyph-and-a-grapheme)
* [Python strings are not UTF-8](https://dev.to/bplevin36/python-strings-are-not-utf-8-2dfj)

<img src="encodings_meme.png" alt="ASCII" width="400"/>
